Notes:

There are two training datasets for the models - the original training split, and the oversampled training split. 

There are also two validation sets - original and oversampled.

So far, the random forest model on the oversampled dataset is the best model.

In [1]:
import pandas as pd
import plotly.express as px
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# !pip install dmba --quiet
from dmba import plotDecisionTree, classificationSummary, regressionSummary
#!pip install dtale --quiet
#!pip install imblearn --quiet
import imblearn
from imblearn.over_sampling import RandomOverSampler
from sklearn import tree
from sklearn.tree import plot_tree

/Users/rakeenahmed/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


no display found. Using non-interactive Agg backend


In [30]:
ksi = pd.read_csv('Datasets/ksi_2021.csv')

In [31]:
ksi.head()

,INDEX,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION
0,3389067,893184,2006,01/01/2006,236,WOODBINE AVE,O CONNOR DR,NaN,Major Arterial,Toronto and East York,...,Yes,Yes,NaN,Yes,NaN,60,Woodbine-Lumsden,60,Woodbine-Lumsden (60),D55
1,3389068,893184,2006,01/01/2006,236,WOODBINE AVE,O CONNOR DR,NaN,Major Arterial,Toronto and East York,...,Yes,Yes,NaN,Yes,NaN,60,Woodbine-Lumsden,60,Woodbine-Lumsden (60),D55
2,3389069,893184,2006,01/01/2006,236,WOODBINE AVE,O CONNOR DR,NaN,Major Arterial,Toronto and East York,...,Yes,Yes,NaN,Yes,NaN,60,Woodbine-Lumsden,60,Woodbine-Lumsden (60),D55
3,3389070,893184,2006,01/01/2006,236,WOODBINE AVE,O CONNOR DR,NaN,Major Arterial,Toronto and East York,...,Yes,Yes,NaN,Yes,NaN,60,Woodbine-Lumsden,60,Woodbine-Lumsden (60),D55
4,3389071,893184,2006,01/01/2006,236,WOODBINE AVE,O CONNOR DR,NaN,Major Arterial,Toronto and East York,...,Yes,Yes,NaN,Yes,NaN,60,Woodbine-Lumsden,60,Woodbine-Lumsden (60),D55


In [32]:
ksi.shape

(17488, 54)

In [33]:
ksi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17488 entries, 0 to 17487
Data columns (total 54 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   INDEX              17488 non-null  int64  
 1   ACCNUM             17488 non-null  int64  
 2   YEAR               17488 non-null  int64  
 3   DATE               17488 non-null  object 
 4   TIME               17488 non-null  int64  
 5   STREET1            17488 non-null  object 
 6   STREET2            15896 non-null  object 
 7   OFFSET             3028 non-null   object 
 8   ROAD_CLASS         17112 non-null  object 
 9   DISTRICT           17453 non-null  object 
 10  WARDNUM            16963 non-null  object 
 11  LATITUDE           17488 non-null  float64
 12  LONGITUDE          17488 non-null  float64
 13  LOCCOORD           17393 non-null  object 
 14  ACCLOC             12038 non-null  object 
 15  TRAFFCTL           17454 non-null  object 
 16  VISIBILITY         174

In [34]:
#changing data types


In [35]:
#looking at null values
ksi.isnull().sum()

INDEX                    0
ACCNUM                   0
YEAR                     0
DATE                     0
TIME                     0
STREET1                  0
STREET2               1592
OFFSET               14460
ROAD_CLASS             376
DISTRICT                35
WARDNUM                525
LATITUDE                 0
LONGITUDE                0
LOCCOORD                95
ACCLOC                5450
TRAFFCTL                34
VISIBILITY              18
LIGHT                    0
RDSFCOND                23
ACCLASS                  7
IMPACTYPE                4
INVTYPE                 14
INVAGE                   0
INJURY                1611
FATAL_NO             16715
INITDIR               4937
VEHTYPE               3033
MANOEUVER             7405
DRIVACT               8666
DRIVCOND              8669
PEDTYPE              14567
PEDACT               14531
PEDCOND              14530
CYCLISTYPE           16749
CYCACT               16758
CYCCOND              16759
PEDESTRIAN           10409
C

In [36]:
#imputing missing values
ksi.fillna('N/A',inplace=True)

In [37]:
#converting column names to lowercase
ksi.columns = [s.strip().lower() for s in ksi.columns]

In [38]:
ksi.columns

Index(['index', 'accnum', 'year', 'date', 'time', 'street1', 'street2',
       'offset', 'road_class', 'district', 'wardnum', 'latitude', 'longitude',
       'loccoord', 'accloc', 'traffctl', 'visibility', 'light', 'rdsfcond',
       'acclass', 'impactype', 'invtype', 'invage', 'injury', 'fatal_no',
       'initdir', 'vehtype', 'manoeuver', 'drivact', 'drivcond', 'pedtype',
       'pedact', 'pedcond', 'cyclistype', 'cycact', 'cyccond', 'pedestrian',
       'cyclist', 'automobile', 'motorcycle', 'truck', 'trsn_city_veh',
       'emerg_veh', 'passenger', 'speeding', 'ag_driv', 'redlight', 'alcohol',
       'disability', 'hood_158', 'neighbourhood_158', 'hood_140',
       'neighbourhood_140', 'division'],
      dtype='object')

In [39]:
#adding a column called intersections
ksi['intersection'] = ksi['street1'].str.cat(ksi['street2'],sep=',')

In [40]:
#setting index as the index
ksi.set_index(ksi.index,inplace=True)

In [41]:
ksi = ksi.drop(columns = ['index','offset','initdir','hood_140','neighbourhood_140','date','fatal_no','accnum','latitude','longitude','year','wardnum','injury'])

In [42]:
ksi.columns

Index(['time', 'street1', 'street2', 'road_class', 'district', 'loccoord',
       'accloc', 'traffctl', 'visibility', 'light', 'rdsfcond', 'acclass',
       'impactype', 'invtype', 'invage', 'vehtype', 'manoeuver', 'drivact',
       'drivcond', 'pedtype', 'pedact', 'pedcond', 'cyclistype', 'cycact',
       'cyccond', 'pedestrian', 'cyclist', 'automobile', 'motorcycle', 'truck',
       'trsn_city_veh', 'emerg_veh', 'passenger', 'speeding', 'ag_driv',
       'redlight', 'alcohol', 'disability', 'hood_158', 'neighbourhood_158',
       'division', 'intersection'],
      dtype='object')

In [43]:
#dropping rows with acclass =  property damage and N/A
#only want acclass = fatal or non-fatal
ksi.drop(ksi[ksi['acclass'] == 'Property Damage Only'].index, inplace=True)
ksi.drop(ksi[ksi['acclass'] == 'N/A'].index, inplace=True)

In [44]:
#specifying predictors and outcome list
outcome = 'acclass'
predictors = [s for s in ksi.columns if s not in outcome]

In [45]:
#assigning predictor and outcome columns to X and y
X = pd.get_dummies(ksi[predictors],drop_first=True)
y = ksi[outcome]

In [46]:
#splitting training and testing dataframes from X and y
from sklearn.model_selection import train_test_split
train_X, valid_X, train_y, valid_y = train_test_split(X,y,test_size=0.3,random_state=1)

In [47]:
ksi.acclass.value_counts()

Non-Fatal Injury    15032
Fatal                2442
Name: acclass, dtype: int64

In [48]:
#checking for class imbalance 
#dataset is extremely imbalanced and will need to be oversampled
train_y.value_counts()

Non-Fatal Injury    10523
Fatal                1708
Name: acclass, dtype: int64

In [49]:
#oversampling for fatal in training data
from imblearn.under_sampling import RandomUnderSampler
undersampler = RandomUnderSampler(random_state=1)
train_X_os, train_y_os = undersampler.fit_resample(train_X,train_y)

In [50]:
train_y_os.value_counts()

Fatal               1708
Non-Fatal Injury    1708
Name: acclass, dtype: int64

In [51]:
#oversampling for fatal in validation data
valid_X_os, valid_y_os = undersampler.fit_resample(valid_X,valid_y)

In [52]:
valid_y_os.value_counts()

Fatal               734
Non-Fatal Injury    734
Name: acclass, dtype: int64

## Decision Tree Classifier

In [24]:
clf_tree = DecisionTreeClassifier(random_state=1)
clf_tree.fit(train_X,train_y)

DecisionTreeClassifier(random_state=1)

In [25]:
classificationSummary(valid_y,clf_tree.predict(valid_X),class_names=['Fatal','Non Fatal'])

Confusion Matrix (Accuracy 0.9208)

          Prediction
   Actual     Fatal Non Fatal
    Fatal       510       224
Non Fatal       191      4318


In [26]:
#fitting tree on oversampled dataset
clf_tree_2 = DecisionTreeClassifier(random_state=1,max_depth=5)
clf_tree_2.fit(train_X_os,train_y_os)

DecisionTreeClassifier(max_depth=5, random_state=1)

In [27]:
classificationSummary(valid_y,clf_tree_2.predict(valid_X),class_names=['Fatal','Non Fatal'])

Confusion Matrix (Accuracy 0.5987)

          Prediction
   Actual     Fatal Non Fatal
    Fatal       627       107
Non Fatal      1997      2512


In [28]:
#classification summary on oversampled validation split
classificationSummary(valid_y_os,clf_tree_2.predict(valid_X_os),class_names=['Fatal','Non Fatal'])

Confusion Matrix (Accuracy 0.7057)

          Prediction
   Actual     Fatal Non Fatal
    Fatal       627       107
Non Fatal       325       409


In [29]:
#import

In [ ]:
#plotting oversampled validation split decision tree
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize = (30,50))
tree.plot_tree(clf_tree_2,fontsize=12,filled=True,feature_names=train_X_os.columns,)
plt.show()

## Neural Network

In [30]:
#trying a neural network model
from sklearn.neural_network import MLPClassifier
neural = MLPClassifier(random_state=1)
neural.fit(train_X,train_y)

MLPClassifier(random_state=1)

In [31]:
classificationSummary(valid_y,neural.predict(valid_X),class_names=['Fatal','Non Fatal'])

Confusion Matrix (Accuracy 0.9163)

          Prediction
   Actual     Fatal Non Fatal
    Fatal       478       256
Non Fatal       183      4326


In [32]:
#fitting neural network on undersampled dataset
neural_2 = MLPClassifier(random_state=1)
neural_2.fit(train_X_os,train_y_os)

MLPClassifier(random_state=1)

In [33]:
classificationSummary(valid_y,neural_2.predict(valid_X),class_names=['Fatal','Non Fatal'])

Confusion Matrix (Accuracy 0.8413)

          Prediction
   Actual     Fatal Non Fatal
    Fatal       527       207
Non Fatal       625      3884


In [34]:
classificationSummary(valid_y_os,neural_2.predict(valid_X_os),class_names=['Fatal','Non Fatal'])

Confusion Matrix (Accuracy 0.7888)

          Prediction
   Actual     Fatal Non Fatal
    Fatal       527       207
Non Fatal       103       631


In [35]:
valid_y.unique()

array(['Fatal', 'Non-Fatal Injury'], dtype=object)

## Random Forest

In [25]:
#random forest
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=500,random_state=1)
forest.fit(train_X,train_y)

RandomForestClassifier(n_estimators=500, random_state=1)

In [119]:
classificationSummary(valid_y,forest.predict(valid_X),class_names=['Fatal','Non Fatal'])

Confusion Matrix (Accuracy 0.9153)

          Prediction
   Actual     Fatal Non Fatal
    Fatal       294       440
Non Fatal         4      4505


In [53]:
#random forest on undersampled dataset
forest_2 = RandomForestClassifier(n_estimators=500,random_state=1)
forest_2.fit(train_X_os,train_y_os)

RandomForestClassifier(n_estimators=500, random_state=1)

In [54]:
#classification summary of original validation split
classificationSummary(valid_y,forest_2.predict(valid_X),class_names=['Fatal','Non Fatal'])

Confusion Matrix (Accuracy 0.7976)

          Prediction
   Actual     Fatal Non Fatal
    Fatal       627       107
Non Fatal       954      3555


In [55]:
#classification summary of acclass=fatal oversampled split
classificationSummary(valid_y_os,forest_2.predict(valid_X_os),class_names=['Fatal','Non Fatal'])

Confusion Matrix (Accuracy 0.8263)

          Prediction
   Actual     Fatal Non Fatal
    Fatal       627       107
Non Fatal       148       586


In [56]:
forest_2.get_params

<bound method BaseEstimator.get_params of RandomForestClassifier(n_estimators=500, random_state=1)>

In [57]:
pd.DataFrame({'Features':forest_2.feature_names_in_,'Importance':forest_2.feature_importances_}).sort_values('Importance',ascending=False).head(15)

,Features,Importance
0,time,0.030627
4490,impactype_Pedestrian Collisions,0.009694
4696,speeding_Yes,0.009508
4472,light_Daylight,0.008725
4688,pedestrian_Yes,0.008277
4697,ag_driv_Yes,0.008242
4434,district_Toronto and East York,0.007852
4692,truck_Yes,0.007185
4445,accloc_N/A,0.006869
4603,drivcond_Unknown,0.006677


In [125]:
#plotting random forest tree
#fig, ax = plt.subplots(figsize = (200,200),nrows=1,ncols=1)
#tree.plot_tree(forest_2.estimators_[0],fontsize=12,filled=True,feature_names=train_X_os.columns)
#plt.show()

Random Forest Gridsearch

In [31]:
param_grid = {'n_estimators':[200,300,500,800,1000],
              'criterion':['gini','entropy'],
              'max_depth':[10,25,50,75,100],
              'min_samples_split':[1,2,3,4,5,6]}

In [ ]:
#from sklearn.model_selection import GridSearchCV
#gridsearch  = GridSearchCV(RandomForestClassifier(random_state=1),param_grid=param_grid,cv=5,n_jobs=-1)
#gridsearch.fit(train_X_os,train_y_os)

In [35]:
gridsearch.best_estimator_

RandomForestClassifier(criterion='entropy', max_depth=50, n_estimators=500,
                       random_state=1)

In [37]:
classificationSummary(valid_y_os,gridsearch.best_estimator_.predict(valid_X_os),class_names=['Fatal','Non Fatal'])

Confusion Matrix (Accuracy 0.8583)

          Prediction
   Actual     Fatal Non Fatal
    Fatal       650        84
Non Fatal       124       610


In [36]:
classificationSummary(valid_y_os,gridsearch.predict(valid_X_os),class_names=['Fatal','Non Fatal'])

Confusion Matrix (Accuracy 0.8583)

          Prediction
   Actual     Fatal Non Fatal
    Fatal       650        84
Non Fatal       124       610


## Boosted Tree

In [43]:
from sklearn.ensemble import AdaBoostClassifier
boost = AdaBoostClassifier(n_estimators=500,random_state=1)
boost.fit(train_X,train_y)

AdaBoostClassifier(n_estimators=500, random_state=1)

In [44]:
classificationSummary(valid_y,boost.predict(valid_X),class_names=['Fatal','Non Fatal'])

Confusion Matrix (Accuracy 0.9174)

          Prediction
   Actual     Fatal Non Fatal
    Fatal       380       354
Non Fatal        79      4430


In [45]:
#boosting on oversampled dataset
boost_2 = AdaBoostClassifier(n_estimators=500,random_state=1)
boost_2.fit(train_X_os,train_y_os)

AdaBoostClassifier(n_estimators=500, random_state=1)

In [46]:
classificationSummary(valid_y,boost_2.predict(valid_X),class_names=['Fatal','Non Fatal'])

Confusion Matrix (Accuracy 0.8009)

          Prediction
   Actual     Fatal Non Fatal
    Fatal       577       157
Non Fatal       887      3622


In [47]:
classificationSummary(valid_y_os,boost_2.predict(valid_X_os),class_names=['Fatal','Non Fatal'])

Confusion Matrix (Accuracy 0.7888)

          Prediction
   Actual     Fatal Non Fatal
    Fatal       577       157
Non Fatal       153       581


## Logistic Regression

In [163]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(random_state=1,C=10,max_iter=10000)
logit.fit(train_X_os,train_y_os)

LogisticRegression(C=10, max_iter=10000, random_state=1)

In [164]:
classificationSummary(valid_y_os,logit.predict(valid_X_os),class_names=['Fatal','Non Fatal'])

Confusion Matrix (Accuracy 0.8556)

          Prediction
   Actual     Fatal Non Fatal
    Fatal       642        92
Non Fatal       120       614


In [165]:
#logistic regression feature importance
pd.DataFrame({'Importance':logit.coef_[0]}).set_index(logit.feature_names_in_).sort_values('Importance',ascending=False).head(15)

,Importance
injury_Major,9.161137
injury_None,6.861853
injury_Minimal,6.832581
injury_Minor,6.667196
"intersection_DUNDAS ST W,BLOOR ST W",3.636261
injury_N/A,3.547932
"intersection_LAWRENCE AVE E,MARKHAM RD",2.938656
impactype_Other,2.832323
"intersection_CARLTON ST,PARLIAMENT ST",2.814368
"intersection_EGLINTON AVE E,PHARMACY AVE",2.732858


## Balanced Random Forest

In [2]:
from imblearn.ensemble import BalancedRandomForestClassifier

In [33]:
balancedforest = BalancedRandomForestClassifier(random_state=1)
balancedforest.fit(train_X_os,train_y_os)

BalancedRandomForestClassifier(random_state=1)

In [34]:
classificationSummary(valid_y_os,balancedforest.predict(valid_X_os),class_names=['Fatal','Non Fatal'])

Confusion Matrix (Accuracy 0.8617)

          Prediction
   Actual     Fatal Non Fatal
    Fatal       646        88
Non Fatal       115       619
